# Introduction

The Purpose of this assigment is to compare 2 different methods for completing the sentiment analysis. I have collected an Amazon Reviews Data Set from Kaggle that contains three million reviews along with their tagged sentiment. The goal is to build two different classifiers using this data and compare their performance. I'll start with NLTKs Naive Bayes classifer and test it against Sklearns and compare the outcome.

In [1]:
%%capture
import nltk
!pip install texthero
nltk.download('vader_lexicon')

In [2]:
%%capture
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import texthero
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np
from sklearn.model_selection import cross_val_predict

  Using cached spacy-2.3.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.5 MB)
  Using cached thinc-7.4.5-cp38-cp38-manylinux2014_x86_64.whl (1.1 MB)
  Using cached numpy-1.21.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.8 MB)
  Using cached blis-0.7.4-cp38-cp38-manylinux2014_x86_64.whl (9.8 MB)
  Using cached cymem-2.0.5-cp38-cp38-manylinux2014_x86_64.whl (35 kB)
  Using cached murmurhash-1.0.5-cp38-cp38-manylinux2014_x86_64.whl (20 kB)
  Using cached srsly-1.0.5-cp38-cp38-manylinux2014_x86_64.whl (186 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached preshed-3.0.5-cp38-cp38-manylinux2014_x86_64.whl (130 kB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached setuptools-58.2.0-py3-none-any.whl (946 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached charset_normalizer-2.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dc-workspace 1.0.0 requires jupytext, which is not installed.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
tensorflow 2.4.1 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
pythonwhat 2.23.1 requires dill~=0.2.7.1, but you have dill 0.3.3 which is incompatible.
pythonwhat 2.23.1 requires jinja2~=2.10, but you have jinja2 3.0.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
data = pd.read_csv('twitterSentiment.csv', header = None, encoding='latin-1')
data.rename(columns={0:"Tag",1:"ID",2:"Date of Tweet", 3:"Ignore", 4:"User", 5:"Tweet"}, inplace = True)
data.head()

,Tag,ID,Date of Tweet,Ignore,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
#0 is negative and 4 is postive sentiment
data['Tag'].value_counts()

0    800000
4    800000
Name: Tag, dtype: int64

In [5]:
# sample the data
SampledDF=data#.sample(frac = .1, weights=data['weights'], random_state = 24)
#SampledDF['Tag'].value_counts()

In [6]:
from texthero import preprocessing
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
#                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace]
SampledDF['Tweet'] = texthero.clean(SampledDF['Tweet'], custom_pipeline)
SampledDF.head()

,Tag,ID,Date of Tweet,Ignore,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
#Sorts the data randomly
TestDF = SampledDF.sample(frac = 1, random_state=24).reset_index(drop=True)
TestDF.head(5)

,Tag,ID,Date of Tweet,Ignore,User,Tweet
0,0,1991032647,Mon Jun 01 06:16:02 PDT 2009,NO_QUERY,LeilaLooo,"just saw angels and demons last night, quality..."
1,0,1976257495,Sat May 30 16:56:01 PDT 2009,NO_QUERY,ddwalker,aaaaaaaaaaaaaand she's gone.
2,0,2228371488,Thu Jun 18 14:21:27 PDT 2009,NO_QUERY,PandaChanda,getting ready to go to my high school's gradua...
3,4,2176174571,Mon Jun 15 03:07:39 PDT 2009,NO_QUERY,ElizabethChingo,bed time. he has a name. ryan
4,0,1559252321,Sun Apr 19 10:39:39 PDT 2009,NO_QUERY,drummer_dan,senses that all is over for the quintuple chances


In [8]:
TestDF['textTag'] = TestDF['Tag'].apply(lambda x: "neg" if x == 0 else "pos")
TestDF.head()

,Tag,ID,Date of Tweet,Ignore,User,Tweet,textTag
0,0,1991032647,Mon Jun 01 06:16:02 PDT 2009,NO_QUERY,LeilaLooo,"just saw angels and demons last night, quality...",neg
1,0,1976257495,Sat May 30 16:56:01 PDT 2009,NO_QUERY,ddwalker,aaaaaaaaaaaaaand she's gone.,neg
2,0,2228371488,Thu Jun 18 14:21:27 PDT 2009,NO_QUERY,PandaChanda,getting ready to go to my high school's gradua...,neg
3,4,2176174571,Mon Jun 15 03:07:39 PDT 2009,NO_QUERY,ElizabethChingo,bed time. he has a name. ryan,pos
4,0,1559252321,Sun Apr 19 10:39:39 PDT 2009,NO_QUERY,drummer_dan,senses that all is over for the quintuple chances,neg


In [9]:
#grab the Independent and Dependent Variables
trainX=TestDF['Tweet']
trainY=TestDF['textTag']
#Verctorize by Term Frequency and Inverse Document Frequency
TFVectorizor = TfidfVectorizer()
xtrainTF = TFVectorizor.fit_transform(trainX)
#inspect the data
print(xtrainTF[:3])

  (0, 64733)	0.16174563235669617
  (0, 608803)	0.13868316431420463
  (0, 661802)	0.15242950887942688
  (0, 83235)	0.1331946798621853
  (0, 560721)	0.2619799151443541
  (0, 124769)	0.12599101967611853
  (0, 426968)	0.2100627197957594
  (0, 232255)	0.2001207762861153
  (0, 601922)	0.08328125936915723
  (0, 601121)	0.20532556660717258
  (0, 102106)	0.18848469502246723
  (0, 301084)	0.10091260328670246
  (0, 370167)	0.27069338315377206
  (0, 608545)	0.1678092046274565
  (0, 233810)	0.3507798487749916
  (0, 500894)	0.30802321685953754
  (0, 448628)	0.1671543031439011
  (0, 360956)	0.1730981962797173
  (0, 183063)	0.321419052114342
  (0, 68781)	0.09953687372003926
  (0, 71171)	0.3087764649816329
  (0, 537430)	0.21903364640703815
  (0, 331378)	0.12784385050836902
  (1, 256261)	0.37578698011464096
  (1, 549328)	0.3072783790591465
  (1, 46635)	0.87427921360347
  (2, 543865)	0.33168978784067715
  (2, 55824)	0.24584292121520104
  (2, 403191)	0.11348208444408868
  (2, 236636)	0.10464321491557545
 

In [10]:
NaiveBayesClassifier = MultinomialNB()
cv = 10

Ypred = cross_val_predict(NaiveBayesClassifier, xtrainTF, trainY, cv = cv)
print(metrics.classification_report(trainY, Ypred))

              precision    recall  f1-score   support

         neg       0.75      0.82      0.78    800000
         pos       0.80      0.73      0.76    800000

    accuracy                           0.77   1600000
   macro avg       0.78      0.77      0.77   1600000
weighted avg       0.78      0.77      0.77   1600000



In [11]:
analyzer = SentimentIntensityAnalyzer()

In [12]:
#Calculates the polarity scores of the data and attaches it to a new column called scores
TestDF['Scores'] = TestDF['Tweet'].apply(lambda review: analyzer.polarity_scores(review))
TestDF['Compound'] = TestDF['Scores'].apply(lambda score_dict: score_dict['compound'])
TestDF.head()

,Tag,ID,Date of Tweet,Ignore,User,Tweet,textTag,Scores,Compound
0,0,1991032647,Mon Jun 01 06:16:02 PDT 2009,NO_QUERY,LeilaLooo,"just saw angels and demons last night, quality...",neg,"{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'comp...",0.6908
1,0,1976257495,Sat May 30 16:56:01 PDT 2009,NO_QUERY,ddwalker,aaaaaaaaaaaaaand she's gone.,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,0,2228371488,Thu Jun 18 14:21:27 PDT 2009,NO_QUERY,PandaChanda,getting ready to go to my high school's gradua...,neg,"{'neg': 0.068, 'neu': 0.697, 'pos': 0.235, 'co...",0.7220
3,4,2176174571,Mon Jun 15 03:07:39 PDT 2009,NO_QUERY,ElizabethChingo,bed time. he has a name. ryan,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,0,1559252321,Sun Apr 19 10:39:39 PDT 2009,NO_QUERY,drummer_dan,senses that all is over for the quintuple chances,neg,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",0.2023


In [13]:
TestDF.loc[TestDF['Compound']>=0,'vaderPred']='pos'
TestDF.loc[TestDF['Compound']<0,'vaderPred']='neg'
TestDF.head()

,Tag,ID,Date of Tweet,Ignore,User,Tweet,textTag,Scores,Compound,vaderPred
0,0,1991032647,Mon Jun 01 06:16:02 PDT 2009,NO_QUERY,LeilaLooo,"just saw angels and demons last night, quality...",neg,"{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'comp...",0.6908,pos
1,0,1976257495,Sat May 30 16:56:01 PDT 2009,NO_QUERY,ddwalker,aaaaaaaaaaaaaand she's gone.,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
2,0,2228371488,Thu Jun 18 14:21:27 PDT 2009,NO_QUERY,PandaChanda,getting ready to go to my high school's gradua...,neg,"{'neg': 0.068, 'neu': 0.697, 'pos': 0.235, 'co...",0.7220,pos
3,4,2176174571,Mon Jun 15 03:07:39 PDT 2009,NO_QUERY,ElizabethChingo,bed time. he has a name. ryan,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
4,0,1559252321,Sun Apr 19 10:39:39 PDT 2009,NO_QUERY,drummer_dan,senses that all is over for the quintuple chances,neg,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",0.2023,pos


In [14]:
print(metrics.classification_report(TestDF['textTag'],TestDF['vaderPred']))

              precision    recall  f1-score   support

         neg       0.81      0.43      0.56    800000
         pos       0.61      0.90      0.73    800000

    accuracy                           0.66   1600000
   macro avg       0.71      0.66      0.64   1600000
weighted avg       0.71      0.66      0.64   1600000

